## Load Libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as utils

from functools import partial as func_partial
from functools import reduce as func_reduce
from operator import mul as op_mul
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from os import cpu_count, path
from time import strftime

from filelock import FileLock

#This class contains DatasetClass and several helper functions
import ClassModule as cm

In [2]:
# Show number of avlaible CPU threads
# With mulithreading this number is twice the number of physical cores
cpu_av = cpu_count()
print("Number of available CPU's: {}".format(cpu_av))

Number of available CPU's: 6


In [3]:
# Set the number CPUS that should be used per trial and dataloader
# If set to 1 number of cucurrent training networking is equal to this number
# In case of training with GPU this will be limited to number of models training simultaneously on GPU
# So number of CPU threads for each trial can be increased 
cpus_per_trial = 2
gpus_per_trial = 0.33

In [4]:
def get_dataloader(train_ds, val_ds, bs):
    
    # We add FileLock here because multiple workers will want to
    # download data, and this may cause overwrites since
    # DataLoader is not threadsafe.
    #with FileLock(path.expanduser("/media/DATA/ML-Notebooks/CNN/Data/.data.lock")):
    dl_train = utils.DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=cpus_per_trial-1)
    dl_val = utils.DataLoader(val_ds, batch_size=bs * 2, shuffle=True, num_workers=cpus_per_trial-1)
    
    return  dl_train, dl_val

## Instance Noise

In [5]:
# https://arxiv.org/abs/1610.04490
INSTANCE_NOISE = True

def add_instance_noise(data, device, std=0.01):
    return data + torch.distributions.Normal(0, std).sample(data.shape).to(device)

## Define the network

In [6]:
class CNN(nn.Module):
    def __init__(self, l1=100, l2=50, l3=25, input_dim=(2,20,20), num_in_features=5):
        super(CNN, self).__init__()
        self.feature_ext = nn.Sequential(
            nn.Conv2d(2,10, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(10,10, kernel_size=5, padding=0),
            nn.ReLU(),  
            nn.MaxPool2d(2),
            nn.Conv2d(10,10, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.Conv2d(10,6, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.flatten = nn.Flatten()
        
        # Gives the number of features after the conv layer
        num_features_after_conv = func_reduce(op_mul, list(self.feature_ext(torch.rand(1, *input_dim)).shape))
        
        self.dense_nn = nn.Sequential(
            nn.Linear(num_features_after_conv + num_in_features, l1),
            nn.ReLU(),
            nn.Linear(l1, l2),
            nn.ReLU(),
            nn.Linear(l2, l3),
            nn.ReLU(),
            nn.Linear(l3,3),
            nn.ReLU()
        )
        
    def forward(self, cluster, clusNumXYEPt):
        cluster = self.feature_ext(cluster)
        x = self.flatten(cluster)
        x = torch.cat([x, clusNumXYEPt], dim=1)
        logits = self.dense_nn(x)
        return logits

## Implement train and validation loop
Data[0] contains an image of of the cell energies and timings. <br>
Data[1] contains all features in a dict. Their shapes have to be changed from [batch_size] to [batch_size,1] for input into linear layers, implemented via function here <br>
Data[2] contains all labels

In [7]:
def train_loop(epoch, dataloader, model, loss_fn, optimizer, device="cpu"):

    size = len(dataloader.dataset)
    running_loss = 0.0
    epoch_steps = 0

    for batch, Data in enumerate(dataloader):
        Clusters = Data[0].to(device)
        Features = cm.unsqueeze_features(Data[1])
        Labels = Data[2]
        
        ClusterProperties = torch.cat([Features["ClusterE"], Features["ClusterPt"], Features["ClusterM02"]
                                      , Features["ClusterM20"], Features["ClusterDist"]], dim=1).to(device)
        #Labels = torch.cat([Labels["PartPID"], dim=1]).to(device)
        Label = Labels["PartPID"].to(device)
        
        if INSTANCE_NOISE:
            Clusters = add_instance_noise(Clusters, device)
        
        # zero parameter gradients
        optimizer.zero_grad()
        
        # prediction and loss
        pred = model(Clusters, ClusterProperties)
        loss = loss_fn(pred, Label.long())
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        epoch_steps += 1
        
        if batch % 100000 == 99999:
            print("[%d, %5d] loss: %.3f" % (epoch + 1, batch + 1,
                                            running_loss / epoch_steps))
            running_loss = 0.0        

In [8]:
def val_loop(epoch, dataloader, model, loss_fn, optimizer, device="cpu"):
    
    val_loss = 0.0
    val_steps = 0
    total = 0
    correct = 0
    size = len(dataloader.dataset)
    
    for batch, Data in enumerate(dataloader):
        with torch.no_grad():
            Clusters = Data[0].to(device)
            Features = cm.unsqueeze_features(Data[1])
            Labels = Data[2]
            ClusterProperties = torch.cat([Features["ClusterE"], Features["ClusterPt"], Features["ClusterM02"]
                                      , Features["ClusterM20"], Features["ClusterDist"]], dim=1).to(device)           
            #Labels = torch.cat([Labels["PartPID"], dim=1]).to(device)
            Label = Labels["PartPID"].to(device)
            
            pred = model(Clusters, ClusterProperties)
            correct += (pred.argmax(1) == Label).type(torch.float).sum().item()

            loss = loss_fn(pred, Label.long())#.item()
            val_loss += loss.cpu().numpy()
            val_steps += 1
    
    with tune.checkpoint_dir(epoch) as checkpoint_dir:
        _path = path.join(checkpoint_dir, "checkpoint")
        torch.save((model.state_dict(), optimizer.state_dict()), _path)
        
    tune.report(loss=(val_loss / val_steps), accuracy= correct / size)

## Implement method for accuracy testing on test set

In [9]:
def test_accuracy(model, device="cpu"):
    
    dataset_test = cm.load_data_test('/media/DATA/ML-Notebooks/CNN/Data/data_test.npz')
    
    dataloader_test = utils.DataLoader(
        dataset_test, batch_size=4, shuffle=False, num_workers=cpu_av-1)
    
    correct = 0
    total = len(dataloader_test.dataset)
    
    with torch.no_grad():
        for batch, Data in enumerate(dataloader_test):
            Clusters = Data[0].to(device)
            Features = cm.unsqueeze_features(Data[1])
            Labels = Data[2]
            ClusterProperties = torch.cat([Features["ClusterE"], Features["ClusterPt"], Features["ClusterM02"]
                                      , Features["ClusterM20"], Features["ClusterDist"]], dim=1).to(device)            
            #Labels = torch.cat([Labels["PartPID"], dim=1]).to(device)
            Label = Labels["PartPID"].to(device)
            
            
            pred = model(Clusters, ClusterProperties)
            correct += (pred.argmax(1) == Label).type(torch.float).sum().item()

    return correct / total

## Implement training routine

In [10]:
def train_model(config, dataloader_train=None, dataloader_test=None, checkpoint_dir=None):
    
    # load model
    model = CNN(config["l1"],config["l2"],config["l3"])
    
    # check for avlaible resource and initialize device
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
    # send model to device
    model.to(device)
    
    # initialise loss function and opptimizer
    loss_fn = F.cross_entropy
    optimizer = torch.optim.Adam(model.parameters(),lr=config["lr"], weight_decay=config["wd"])
    
    # check whether checkpoint is available
    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            path.join(checkpoint_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
        
    # load dataset
    dataset_train = cm.load_data_train('/media/DATA/ML-Notebooks/CNN/Data/data_train.npz')
    
    # split trainset in train and validation subsets
    test_abs = int(len(dataset_train) * 0.8)
    subset_train, subset_val = utils.random_split(
        dataset_train, [test_abs, len(dataset_train) - test_abs])

    # get dataloaders 
    dataloader_train, dataloader_val = get_dataloader(subset_train, subset_val, int(config["batch_size"]))
                                                      
    for epoch in range(100):
        train_loop(epoch, dataloader_train, model, loss_fn, optimizer, device=device)
        val_loop(epoch, dataloader_val, model, loss_fn, optimizer, device=device)                                              
    
    print("Finished Training")

## Setup all Ray Tune functionality and start training

In [11]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=1):
    
    # Setup hyperparameter-space to search
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 8)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 8)),
        "l3": tune.sample_from(lambda _: 2 ** np.random.randint(2, 8)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "wd": tune.loguniform(1e-5, 1e-3),
        "batch_size": tune.choice([16, 32, 64, 128, 256])
    }

    # Init the scheduler
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    
    # Init the Reporter
    reporter = CLIReporter(
        parameter_columns=["l1", "l2", "l3", "lr","wd", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    
    #Get Current date and time
    timestr = strftime("%Y_%m_%d-%H:%M:%S")
    name = "ASHA-" + timestr
    
    #Load the dataset for use in shared memory
    #data = cm.load_data_train('/media/DATA/ML-Notebooks/CNN/Data/data_train.npz')
    #test_abs = int(len(data) * 0.8)
    #subset_train, subset_val = utils.random_split(
    #    data, [test_abs, len(data) - test_abs])

    # get dataloaders 
    #dataloader_train, dataloader_val = get_dataloader(subset_train, subset_val, 64)
    
    # Init the run method
    result = tune.run(
        func_partial(train_model),
        #tune.with_parameters(train_model, dataset_train=data),
        #tune.with_parameters(train_model, dataloader_train=dataloader_train, dataloader_val=dataloader_val),
        name = name,
        resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        local_dir = "./Ray_Results",
        scheduler=scheduler,
        progress_reporter=reporter)
    
    # Find best trial and use it on the testset
    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(best_trial.last_result["accuracy"]))
    
    best_trained_model = CNN(best_trial.config["l1"], best_trial.config["l2"], best_trial.config["l3"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)
    
    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)
    
    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))

In [12]:
main(num_samples=10, max_num_epochs=30, gpus_per_trial=gpus_per_trial)

/home/j_hone04/anaconda3/envs/cnn-env/lib/python3.9/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)
2021-09-30 16:58:11,537	WARNING experiment.py:295 -- No name detected on trainable. Using DEFAULT.
2021-09-30 16:58:11,544	INFO registry.py:66 -- Detected unknown callable for trainable. Converting to class.


== Status ==
Memory usage on this node: 4.1/31.2 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/6 CPUs, 0.33/1 GPUs, 0.0/6.1 GiB heap, 0.0/3.05 GiB objects (0.0/1.0 accelerator_type:GTX)
Result logdir: /media/DATA/ML-Notebooks/CNN/Ray_Results/ASHA-2021_09_30-16:58:09
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-------+------+------+------+-------------+-------------+--------------+
| Trial name          | status   | loc   |   l1 |   l2 |   l3 |          lr |          wd |   batch_size |
|---------------------+----------+-------+------+------+------+-------------+-------------+--------------|
| DEFAULT_d4b93_00000 | RUNNING  |       |    4 |   64 |    4 | 0.00708603  | 0.000222663 |           32 |
| DEFAULT_d4b93_00001 | PENDING  |       |   64 |   16 |   16 | 0.0182425   | 6.57155e-05 |           32 |
| DEFAULT_d4b93_00002 | PENDI

(pid=14503) [1, 200000] loss: 0.549
(pid=14507) [2, 100000] loss: 1.098
(pid=14508) [2, 100000] loss: 1.077
Result for DEFAULT_d4b93_00002:
  accuracy: 0.358641604383116
  date: 2021-09-30_17-35-32
  done: true
  experiment_id: f72d1ef3bbc94abfa197409fb97aaa93
  hostname: JanHonermann
  iterations_since_restore: 1
  loss: 1.0986120700836182
  node_ip: 128.176.166.36
  pid: 14503
  should_checkpoint: true
  time_since_restore: 2239.9014027118683
  time_this_iter_s: 2239.9014027118683
  time_total_s: 2239.9014027118683
  timestamp: 1633016132
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d4b93_00002
  
== Status ==
Memory usage on this node: 22.7/31.2 GiB
Using AsyncHyperBand: num_stopped=1
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: -1.0986120700836182
Resources requested: 6.0/6 CPUs, 0.99/1 GPUs, 0.0/6.1 GiB heap, 0.0/3.05 GiB objects (0.0/1.0 accelerator_type:GTX)
Result logdir: /media/DATA/ML-Notebooks/CNN/Ray_R

Result for DEFAULT_d4b93_00003:
  accuracy: 0.3589175460795402
  date: 2021-09-30_17-49-28
  done: true
  experiment_id: 3004a0d3a4934ef0929d696b8313c006
  hostname: JanHonermann
  iterations_since_restore: 1
  loss: 1.098612070415986
  node_ip: 128.176.166.36
  pid: 14506
  should_checkpoint: true
  time_since_restore: 834.9950065612793
  time_this_iter_s: 834.9950065612793
  time_total_s: 834.9950065612793
  timestamp: 1633016968
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d4b93_00003
  
== Status ==
Memory usage on this node: 22.7/31.2 GiB
Using AsyncHyperBand: num_stopped=2
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: -1.087641557293153 | Iter 1.000: -1.0986120700974684
Resources requested: 6.0/6 CPUs, 0.99/1 GPUs, 0.0/6.1 GiB heap, 0.0/3.05 GiB objects (0.0/1.0 accelerator_type:GTX)
Result logdir: /media/DATA/ML-Notebooks/CNN/Ray_Results/ASHA-2021_09_30-16:58:09
Number of trials: 10/10 (6 PENDING, 3 RUNNING, 1 TERMINATED)
+---

(pid=14505) [1, 200000] loss: 0.559
(pid=14507) [4, 100000] loss: 1.099
(pid=14508) [4, 100000] loss: 1.078
Result for DEFAULT_d4b93_00004:
  accuracy: 0.3587287438661973
  date: 2021-09-30_18-26-12
  done: false
  experiment_id: 7e7ee7e3ae054368b2f034cb56dd661f
  hostname: JanHonermann
  iterations_since_restore: 1
  loss: 1.0986120700836182
  node_ip: 128.176.166.36
  pid: 14505
  should_checkpoint: true
  time_since_restore: 2203.0121731758118
  time_this_iter_s: 2203.0121731758118
  time_total_s: 2203.0121731758118
  timestamp: 1633019172
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d4b93_00004
  
== Status ==
Memory usage on this node: 22.2/31.2 GiB
Using AsyncHyperBand: num_stopped=2
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: -1.087641557293153 | Iter 1.000: -1.0986120700836182
Resources requested: 6.0/6 CPUs, 0.99/1 GPUs, 0.0/6.1 GiB heap, 0.0/3.05 GiB objects (0.0/1.0 accelerator_type:GTX)
Result logdir: /media/DATA/ML-Not

...

(pid=21958) [23, 100000] loss: 1.011
(pid=21958) [23, 200000] loss: 0.505
Result for DEFAULT_d4b93_00009:
  accuracy: 0.47818517503235963
  date: 2021-10-01_17-04-36
  done: false
  experiment_id: f779db1531284ba4b4101762cc933d6e
  hostname: JanHonermann
  iterations_since_restore: 23
  loss: 1.0170593684117415
  node_ip: 128.176.166.36
  pid: 21958
  should_checkpoint: true
  time_since_restore: 44205.99614882469
  time_this_iter_s: 1753.7605702877045
  time_total_s: 44205.99614882469
  timestamp: 1633100676
  timesteps_since_restore: 0
  training_iteration: 23
  trial_id: d4b93_00009
  
== Status ==
Memory usage on this node: 10.5/31.2 GiB
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 16.000: -1.0224705044886437 | Iter 8.000: -1.028652109918929 | Iter 4.000: -1.0288182652683506 | Iter 2.000: -1.053732617376248 | Iter 1.000: -1.0768198621900458
Resources requested: 2.0/6 CPUs, 0.33/1 GPUs, 0.0/6.1 GiB heap, 0.0/3.05 GiB objects (0.0/1.0 accelerator_type:GTX)
Result logdir: /media/

(pid=21958) [26, 100000] loss: 1.010
(pid=21958) [26, 200000] loss: 0.505
Result for DEFAULT_d4b93_00009:
  accuracy: 0.4848740380618001
  date: 2021-10-01_18-32-12
  done: false
  experiment_id: f779db1531284ba4b4101762cc933d6e
  hostname: JanHonermann
  iterations_since_restore: 26
  loss: 1.0096031135142687
  node_ip: 128.176.166.36
  pid: 21958
  should_checkpoint: true
  time_since_restore: 49461.469949007034
  time_this_iter_s: 1752.7092506885529
  time_total_s: 49461.469949007034
  timestamp: 1633105932
  timesteps_since_restore: 0
  training_iteration: 26
  trial_id: d4b93_00009
  
== Status ==
Memory usage on this node: 10.6/31.2 GiB
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 16.000: -1.0224705044886437 | Iter 8.000: -1.028652109918929 | Iter 4.000: -1.0288182652683506 | Iter 2.000: -1.053732617376248 | Iter 1.000: -1.0768198621900458
Resources requested: 2.0/6 CPUs, 0.33/1 GPUs, 0.0/6.1 GiB heap, 0.0/3.05 GiB objects (0.0/1.0 accelerator_type:GTX)
Result logdir: /media

(pid=21958) [29, 100000] loss: 1.011
(pid=21958) [29, 200000] loss: 0.505
Result for DEFAULT_d4b93_00009:
  accuracy: 0.4816081228521479
  date: 2021-10-01_19-59-55
  done: false
  experiment_id: f779db1531284ba4b4101762cc933d6e
  hostname: JanHonermann
  iterations_since_restore: 29
  loss: 1.0136393359487337
  node_ip: 128.176.166.36
  pid: 21958
  should_checkpoint: true
  time_since_restore: 54724.85769701004
  time_this_iter_s: 1757.3190925121307
  time_total_s: 54724.85769701004
  timestamp: 1633111195
  timesteps_since_restore: 0
  training_iteration: 29
  trial_id: d4b93_00009
  
== Status ==
Memory usage on this node: 10.5/31.2 GiB
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 16.000: -1.0224705044886437 | Iter 8.000: -1.028652109918929 | Iter 4.000: -1.0288182652683506 | Iter 2.000: -1.053732617376248 | Iter 1.000: -1.0768198621900458
Resources requested: 2.0/6 CPUs, 0.33/1 GPUs, 0.0/6.1 GiB heap, 0.0/3.05 GiB objects (0.0/1.0 accelerator_type:GTX)
Result logdir: /media/D

2021-10-01 20:29:07,691	INFO tune.py:561 -- Total run time: 99056.16 seconds (99055.43 seconds for the tuning loop).


== Status ==
Memory usage on this node: 4.4/31.2 GiB
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 16.000: -1.0224705044886437 | Iter 8.000: -1.028652109918929 | Iter 4.000: -1.0288182652683506 | Iter 2.000: -1.053732617376248 | Iter 1.000: -1.0768198621900458
Resources requested: 0/6 CPUs, 0/1 GPUs, 0.0/6.1 GiB heap, 0.0/3.05 GiB objects (0.0/1.0 accelerator_type:GTX)
Result logdir: /media/DATA/ML-Notebooks/CNN/Ray_Results/ASHA-2021_09_30-16:58:09
Number of trials: 10/10 (10 TERMINATED)
+---------------------+------------+-------+------+------+------+-------------+-------------+--------------+----------+------------+----------------------+
| Trial name          | status     | loc   |   l1 |   l2 |   l3 |          lr |          wd |   batch_size |     loss |   accuracy |   training_iteration |
|---------------------+------------+-------+------+------+------+-------------+-------------+--------------+----------+------------+----------------------|
| DEFAULT_d4b93_00000 | TERMINATED